In [1]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By

import time

import pandas as pd

from selenium.webdriver.common.keys import Keys

In [2]:
def read_cas_data(csv_filepath="src/smiles_pine.csv"):
    df = pd.read_csv(csv_filepath)
    df = df[["cas", "name"]]
    return df 

In [3]:
def accept_cookies(driver,url):
    driver.get(url)
    time.sleep(3)
    cookies = driver.find_elements(By.ID, 'onetrust-accept-btn-handler')
    cookies[0].click()
    time.sleep(1)
    return

In [4]:


def get_vapour_pressures(cas,name, driver,url, byname=False):
    if byname:
        key = name
    else:
        key = cas
    driver.get(url)
    time.sleep(3)
    search = driver.find_elements(By.ID, "input-left-icon")
    search[0].send_keys(key)
    time.sleep(5)
    search[0].send_keys(Keys.RETURN)
    time.sleep(3)
    found_header = driver.find_element(By.CLASS_NAME, "found-header").text
    if found_header != "Found 1 result":
        if found_header == "Found 0 results":
            if not byname:
                return get_vapour_pressures(cas, name, driver,url,byname=True)
            return None, None, None
        driver.find_element(By.CLASS_NAME, "search-list").click()
        time.sleep(1)

    properties_tab = driver.find_element(By.ID, "tab1")
    properties_tab.click()
    time.sleep(1)
    acd_tab = driver.find_element(By.ID, "accordion-acd/labs-title")
    acd_tab.click()
    time.sleep(1)
    vapour_pressure_label = driver.find_element(By.XPATH, '//tr[th[text()="Vapour pressure"]]/td')
    vapour_pressure_value = vapour_pressure_label.text  # Get the text from the corresponding <td>

    mol_name = driver.find_element(By.ID, "cmp-title-label").text
    current_url = driver.current_url
    return mol_name, vapour_pressure_value, current_url



In [5]:
def main():
    df = read_cas_data()
    options = webdriver.ChromeOptions()
    options.page_load_strategy = 'normal'
    driver = webdriver.Chrome(options=options)
    url="https://www.chemspider.com/"
    accept_cookies(driver,url)

    results_df = pd.DataFrame({
        "fetched_name":[],
        "vapour_pressure":[],
        "url":[]
    })
    
    for i in df.index:
        row = list(df.loc[i])
        cas_num = row[0]
        name = row[1]
        print(f" {i} Fetching {cas_num} ({name})", end=" ")
        fetched_name,vapour_pressure_value, current_url =get_vapour_pressures(cas_num,name, driver, url)
        print(f"={fetched_name} VP: {vapour_pressure_value}")
        results_df.loc[len(results_df)] = [fetched_name, vapour_pressure_value,current_url]

    driver.quit()

    return results_df

In [6]:
results_df = main()

 0 Fetching 80-56-8 (alpha-Pinene) =(+)-α-Pinene VP: 3.5±0.1 mmHg at 25°C
 1 Fetching 13466-78-9 (delta-3-Carene) =carene VP: 1.9±0.1 mmHg at 25°C
 2 Fetching 1139-30-6 (Caryophyllene oxide) =(−)-β-Caryophyllene epoxide VP: 0.0±0.6 mmHg at 25°C
 3 Fetching 483-76-1 (delta-Cadinene) =d-Cadinene VP: 0.0±0.3 mmHg at 25°C
 4 Fetching 79-92-5 (Camphene) =Camphene VP: 3.4±0.1 mmHg at 25°C
 5 Fetching 87-44-5 (beta-Caryophyllene) =(−)-β-caryophyllene VP: 0.0±0.3 mmHg at 25°C
 6 Fetching 31983-22-9 (alpha-Muurolene) =MPTP VP: 0.0±0.5 mmHg at 25°C
 7 Fetching 481-34-5 (alpha-Cadinol) =alpha-cadinol VP: 0.0±1.4 mmHg at 25°C
 8 Fetching 586-62-9 (Terpinolene) =TERPINOLENE VP: 1.1±0.2 mmHg at 25°C
 9 Fetching 127-91-3 (beta-Pinene) =(±)-β-Pinene VP: 2.4±0.1 mmHg at 25°C
 10 Fetching 123-35-3 (Myrcene) =β-Myrcene VP: 2.3±0.1 mmHg at 25°C
 11 Fetching 30021-74-0 (gamma-Muurolene) =(1R,4aR,8aS)-1-Isopropyl-7-methyl-4-methylene-1,2,3,4,4a,5,6,8a-octahydronaphthalene VP: 0.0±0.3 mmHg at 25°C
 12 Fetchi

In [7]:
results_df

,fetched_name,vapour_pressure,url
0,(+)-α-Pinene,3.5±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
1,carene,1.9±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
2,(−)-β-Caryophyllene epoxide,0.0±0.6 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
3,d-Cadinene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
4,Camphene,3.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
5,(−)-β-caryophyllene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
6,MPTP,0.0±0.5 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
7,alpha-cadinol,0.0±1.4 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
8,TERPINOLENE,1.1±0.2 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....
9,(±)-β-Pinene,2.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....


In [ ]:
#results_df.to_csv("scaped_vp.csv")

In [12]:
results_df['vapour_pressure2'] = results_df["vapour_pressure"].apply(lambda x: x.split("±")[0])

In [13]:
results_df

,fetched_name,vapour_pressure,url,vapour_pressure2
0,(+)-α-Pinene,3.5±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,3.5
1,carene,1.9±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,1.9
2,(−)-β-Caryophyllene epoxide,0.0±0.6 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
3,d-Cadinene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
4,Camphene,3.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,3.4
5,(−)-β-caryophyllene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
6,MPTP,0.0±0.5 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
7,alpha-cadinol,0.0±1.4 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
8,TERPINOLENE,1.1±0.2 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,1.1
9,(±)-β-Pinene,2.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,2.4


In [ ]:
#results_df.to_csv("scaped_vp.csv")

In [18]:
ny_df = pd.concat([read_cas_data(),results_df], axis=1 )
ny_df

,cas,name,fetched_name,vapour_pressure,url,vapour_pressure2
0,80-56-8,alpha-Pinene,(+)-α-Pinene,3.5±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,3.5
1,13466-78-9,delta-3-Carene,carene,1.9±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,1.9
2,1139-30-6,Caryophyllene oxide,(−)-β-Caryophyllene epoxide,0.0±0.6 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
3,483-76-1,delta-Cadinene,d-Cadinene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
4,79-92-5,Camphene,Camphene,3.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,3.4
5,87-44-5,beta-Caryophyllene,(−)-β-caryophyllene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
6,31983-22-9,alpha-Muurolene,MPTP,0.0±0.5 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
7,481-34-5,alpha-Cadinol,alpha-cadinol,0.0±1.4 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
8,586-62-9,Terpinolene,TERPINOLENE,1.1±0.2 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,1.1
9,127-91-3,beta-Pinene,(±)-β-Pinene,2.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,2.4


In [21]:
urls = results_df[["fetched_name","url","vapour_pressure2"]].loc[[6,11,19,20,31,36,38,45,51]]["url"]
for i in [6,11,19,20,31,36,38,45,51]:
    print(f"{i} {urls[i]}")

6 https://www.chemspider.com/Chemical-Structure.1346.html
11 https://www.chemspider.com/Chemical-Structure.4937567.html
19 https://www.chemspider.com/Chemical-Structure.71367.html
20 https://www.chemspider.com/Chemical-Structure.2656.html
31 https://www.chemspider.com/Chemical-Structure.2341398.html
36 https://www.chemspider.com/Chemical-Structure.29273044.html
38 https://www.chemspider.com/Chemical-Structure.56330.html
45 https://www.chemspider.com/Chemical-Structure.454791.html
51 https://www.chemspider.com/Chemical-Structure.34987048.html


In [ ]:
[6, "α-Muurolene","0.0±0.3 mmHg at 25°C","https://www.chemspider.com/Chemical-Structure.10209840.html"]

In [22]:
results_df.loc[6] = ["α-Muurolene","0.0±0.3 mmHg at 25°C","https://www.chemspider.com/Chemical-Structure.10209840.html", 0.0]

In [23]:
results_df

,fetched_name,vapour_pressure,url,vapour_pressure2
0,(+)-α-Pinene,3.5±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,3.5
1,carene,1.9±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,1.9
2,(−)-β-Caryophyllene epoxide,0.0±0.6 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
3,d-Cadinene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
4,Camphene,3.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,3.4
5,(−)-β-caryophyllene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
6,α-Muurolene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
7,alpha-cadinol,0.0±1.4 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
8,TERPINOLENE,1.1±0.2 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,1.1
9,(±)-β-Pinene,2.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,2.4


In [ ]:
#results_df.to_csv("scaped_vp.csv")

In [27]:
ny_df = pd.concat([read_cas_data(),results_df], axis=1 )
ny_df["vapour_pressure2"] = ny_df["vapour_pressure2"].astype(float)
ny_df

,cas,name,fetched_name,vapour_pressure,url,vapour_pressure2
0,80-56-8,alpha-Pinene,(+)-α-Pinene,3.5±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,3.5
1,13466-78-9,delta-3-Carene,carene,1.9±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,1.9
2,1139-30-6,Caryophyllene oxide,(−)-β-Caryophyllene epoxide,0.0±0.6 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
3,483-76-1,delta-Cadinene,d-Cadinene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
4,79-92-5,Camphene,Camphene,3.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,3.4
5,87-44-5,beta-Caryophyllene,(−)-β-caryophyllene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
6,31983-22-9,alpha-Muurolene,α-Muurolene,0.0±0.3 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
7,481-34-5,alpha-Cadinol,alpha-cadinol,0.0±1.4 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,0.0
8,586-62-9,Terpinolene,TERPINOLENE,1.1±0.2 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,1.1
9,127-91-3,beta-Pinene,(±)-β-Pinene,2.4±0.1 mmHg at 25°C,https://www.chemspider.com/Chemical-Structure....,2.4


LogP: 2.9987000000000013


ModuleNotFoundError: No module named 'chempy.properties.water_solubility'

In [ ]:
def read_cas_data(csv_filepath="src/smiles_pine.csv"):
    df = pd.read_csv(csv_filepath)
    df = df[["cas", "name"]]
    return df 

In [8]:

"""options = webdriver.ChromeOptions()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
url="https://www.chemspider.com/"
accept_cookies(driver,url)

get_vapour_pressures_from_cas("25548-04-3", driver,url)"""


'options = webdriver.ChromeOptions()\noptions.page_load_strategy = \'normal\'\ndriver = webdriver.Chrome(options=options)\nurl="https://www.chemspider.com/"\naccept_cookies(driver,url)\n\nget_vapour_pressures_from_cas("25548-04-3", driver,url)'

In [9]:
"""d"river.quit()""""""

SyntaxError: incomplete input (1539533678.py, line 1)

In [58]:
import re

In [66]:
def detect_water_sol(text):
    # Pattern to extract water solubility values
    # From [logP, stucture, experimental]
    pattern = r"Water Solubility at 25 deg C \(mg/L\):\s+([\d.]+)|Wat Sol \(v1\.01 est\)\s*=\s*([\d.]+) mg/L|Wat Sol \(Exper\. database match\)\s*=\s*([\d.]+)"


    # Find matches
    matches = re.findall(pattern, text)
    labels = ["sol. from logP", "sol. from structure", "sol. experiental"]
    # Extract and format results
    water_sol_values = [None for i in range(3)]
    for match in matches:
        # Each match has two groups; one of them will be filled
        for i, val in enumerate(match):
            if val != '' and water_sol_values[i]==None:
                water_sol_values[i]=float(val)
            elif val != '' and water_sol_values[i]!=None:
                print(f"Double value for index {labels[i]} detected")
            
        #value = next(v for v in match if v)  # Get the non-empty group
        #water_sol_values.append(float(value))  # Convert to float for numerical processing
    return water_sol_values
  

In [67]:
def get_add_data(url, driver):
    driver.get(url)
    time.sleep(3)
    
    properties_tab = driver.find_element(By.ID, "tab1")
    properties_tab.click()
    time.sleep(1)
    acd_tab = driver.find_element(By.ID, "accordion-acd/labs-title")
    acd_tab.click()
    time.sleep(1)
    logP_label = driver.find_element(By.XPATH, '//tr[th[text()="ACD/LogP"]]/td')
    logP_value = logP_label.text  # Get the text from the corresponding <td>

    

    mol_name = driver.find_element(By.ID, "cmp-title-label").text
    current_url = driver.current_url

    legacy_btns = driver.find_elements(By.ID, "button-download")
    #for i in legacy_btns:
        #print(i.get_attribute('outerHTML'))
    time.sleep(1)
    # Get the parent element using XPath
    legacy_url = legacy_btns[1].find_element(By.XPATH, "..").get_attribute('href')
    #print(legacy_url)
    driver.get(legacy_url)
    time.sleep(3)
    driver.find_element(By.ID, "ctl00_ctl00_ContentSection_ContentPlaceHolder1_RecordViewTabDetailsControl_prop").click()
    time.sleep(1)
    driver.find_element(By.ID, "ui-id-7").click()
    time.sleep(1)
    predicted = driver.find_element(By.TAG_NAME, "pre")
    #print(predicted.text)
    water_sol_values = detect_water_sol(predicted.text)
    #ui-id-7
    #legacy_btns[1].click()
    time.sleep(5)
    return mol_name, logP_value, legacy_url, water_sol_values

In [52]:
options = webdriver.ChromeOptions()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
url="https://www.chemspider.com/Chemical-Structure.10209840.html"
accept_cookies(driver,url)
get_add_data(url, driver)
driver.quit()

https://legacy.chemspider.com/Chemical-Structure.10209840.html
                        
 Log Octanol-Water Partition Coef (SRC):
    Log Kow (KOWWIN v1.67 estimate) =  6.19

 Boiling Pt, Melting Pt, Vapor Pressure Estimations (MPBPWIN v1.42):
    Boiling Pt (deg C):  258.98  (Adapted Stein & Brown method)
    Melting Pt (deg C):  26.16  (Mean or Weighted MP)
    VP(mm Hg,25 deg C):  0.0271  (Modified Grain method)
    Subcooled liquid VP: 0.0278 mm Hg (25 deg C, Mod-Grain method)

 Water Solubility Estimate from Log Kow (WSKOW v1.41):
    Water Solubility at 25 deg C (mg/L):  0.06276
       log Kow used: 6.19 (estimated)
       no-melting pt equation used

 Water Sol Estimate from Fragments:
    Wat Sol (v1.01 est) =  0.5123 mg/L

 ECOSAR Class Program (ECOSAR v0.99h):
    Class(es) found:
       Neutral Organics

 Henrys Law Constant (25 deg C) [HENRYWIN v3.10]:
   Bond Method :   8.15E-001  atm-m3/mole
   Group Method:   Incomplete
 Henrys LC [VP/WSol estimate using EPI values]:  1.1

In [ ]:
 """Water Solubility Estimate from Log Kow (WSKOW v1.41):
    Water Solubility at 25 deg C (mg/L):  0.06276
       log Kow used: 6.19 (estimated)
       no-melting pt equation used

 Water Sol Estimate from Fragments:
    Wat Sol (v1.01 est) =  0.5123 mg/L"""

In [ ]:
def read_extra(df):
    options = webdriver.ChromeOptions()
    options.page_load_strategy = 'normal'
    driver = webdriver.Chrome(options=options)
    url="https://www.chemspider.com/"
    accept_cookies(driver,url)
    #mol_name, logP_value, legacy_url, water_sol_values
    results_df = pd.DataFrame({
        "mol_name":[],
        "logP_value":[],
        "legacy_url":[],
        "water_sol_from_logP":[],
        "water_sol_from_structure":[],
        "water_sol_experimental":[]
    })
    
    for i in df.index:
        row = list(df[["cas", "name", "url"]].loc[i])
        cas_num = row[0]
        name = row[1]
        url = row[2]
        print(f" {i} Fetching {cas_num} ({name})", end=" ")
        fetched_name, logP_value, legacy_url, water_sol_values =get_add_data(url, driver) 
        print(f"={fetched_name} logP: {logP_value} VP: {water_sol_values}")
        results_df.loc[len(results_df)] = [fetched_name, logP_value,legacy_url,water_sol_values[0], water_sol_values[1], water_sol_values[2] ]

    driver.quit()

    return results_df

In [69]:
soldata_df = read_extra(ny_df)

C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cas_num = row[0]
C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = row[1]
C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = row[2]


 0 Fetching 80-56-8 (alpha-Pinene) =(+)-α-Pinene logP: 4.37 VP: [4.071, 3.4834, 2.49]
 1 Fetching 13466-78-9 (delta-3-Carene) 

C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cas_num = row[0]
C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = row[1]
C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = row[2]


=carene logP: 4.37 VP: [4.581, 3.4834, None]
 2 Fetching 1139-30-6 (Caryophyllene oxide) 

C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df.loc[len(results_df)] = [fetched_name, logP_value,legacy_url,water_sol_values[0], water_sol_values[1], water_sol_values[2] ]
C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cas_num = row[0]
C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will

=(−)-β-Caryophyllene epoxide logP: 4.57 VP: [2.21, 4.626, None]
 3 Fetching 483-76-1 (delta-Cadinene) 

C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df.loc[len(results_df)] = [fetched_name, logP_value,legacy_url,water_sol_values[0], water_sol_values[1], water_sol_values[2] ]
C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cas_num = row[0]
C:\Users\paulj\AppData\Local\Temp\ipykernel_23508\2819980673.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will

KeyboardInterrupt: 

In [65]:
text="""
Log Octanol-Water Partition Coef (SRC):
    Log Kow (KOWWIN v1.67 estimate) =  4.27
    Log Kow (Exper. database match) =  4.83
       Exper. Ref:  Li,J & Perdue,EM (1995)
    Log Kow (Exper. database match) =  4.48
       Exper. Ref:  Griffin,S et al. (1999)
    Log Kow (Exper. database match) =  4.44
       Exper. Ref:  Griffin,S et al. (1999)

 Boiling Pt, Melting Pt, Vapor Pressure Estimations (MPBPWIN v1.42):
    Boiling Pt (deg C):  157.25  (Adapted Stein & Brown method)
    Melting Pt (deg C):  -13.62  (Mean or Weighted MP)
    VP(mm Hg,25 deg C):  4.02  (Mean VP of Antoine & Grain methods)
    MP  (exp database):  -62 deg C
    BP  (exp database):  155.5 deg C
    VP  (exp database):  4.75E+00 mm Hg at 25 deg C

 Water Solubility Estimate from Log Kow (WSKOW v1.41):
    Water Solubility at 25 deg C (mg/L):  4.071
    Water Solubility at 25 deg C (mg/L):  4.071
       log Kow used: 4.44 (expkow database)
       no-melting pt equation used
     Water Sol (Exper. database match) =  2.49 mg/L (25 deg C)
        Exper. Ref:  LI,J & PERDUE,EM (1995)

 Water Sol Estimate from Fragments:
    
    Wat Sol (Exper. database match) =  2.49
       Exper. Ref:  LI,J & PERDUE,EM (1995)

 ECOSAR Class Program (ECOSAR v0.99h):
    Class(es) found:
       Neutral Organics

 Henrys Law Constant (25 deg C) [HENRYWIN v3.10]:
   Bond Method :   1.07E-001  atm-m3/mole
   Group Method:   Incomplete
   Exper Database: 2.94E-01  atm-m3/mole
 Henrys LC [VP/WSol estimate using EPI values]:  1.770E-001 atm-m3/mole

 Log Octanol-Air Partition Coefficient (25 deg C) [KOAWIN v1.10]:
  Log Kow used:  4.44  (exp database)
  Log Kaw used:  1.080  (exp database)
      Log Koa (KOAWIN v1.10 estimate):  3.360
      Log Koa (experimental database):  None

 Probability of Rapid Biodegradation (BIOWIN v4.10):
   Biowin1 (Linear Model)         :   0.4988
   Biowin2 (Non-Linear Model)     :   0.3432
 Expert Survey Biodegradation Results:
   Biowin3 (Ultimate Survey Model):   2.6860  (weeks-months)
   Biowin4 (Primary Survey Model) :   3.4977  (days-weeks  )
 MITI Biodegradation Probability:
   Biowin5 (MITI Linear Model)    :   0.4462
   Biowin6 (MITI Non-Linear Model):   0.3302
 Anaerobic Biodegradation Probability:
   Biowin7 (Anaerobic Linear Model):  0.0285
 Ready Biodegradability Prediction:   NO

 Hydrocarbon Biodegradation (BioHCwin v1.01):
     LOG BioHC Half-Life (days) :   1.6807
     BioHC Half-Life (days)     :  47.9347

 Sorption to aerosols (25 Dec C)[AEROWIN v1.00]:
  Vapor pressure (liquid/subcooled):  633 Pa (4.75 mm Hg)
  Log Koa (Koawin est  ): 3.360
   Kp (particle/gas partition coef. (m3/ug)):
       Mackay model           :  4.74E-009 
       Octanol/air (Koa) model:  5.62E-010 
   Fraction sorbed to airborne particulates (phi):
       Junge-Pankow model     :  1.71E-007 
       Mackay model           :  3.79E-007 
       Octanol/air (Koa) model:  4.5E-008 

 Atmospheric Oxidation (25 deg C) [AopWin v1.92]:
   Hydroxyl Radicals Reaction:
      OVERALL OH Rate Constant =  90.7477 E-12 cm3/molecule-sec
      Half-Life =     0.118 Days (12-hr day; 1.5E6 OH/cm3)
      Half-Life =     1.414 Hrs
   Ozone Reaction:
      OVERALL Ozone Rate Constant =    43.000000 E-17 cm3/molecule-sec
      Half-Life =     0.027 Days (at 7E11 mol/cm3)
      Half-Life =     38.378 Min
   Reaction With Nitrate Radicals May Be Important!
   Fraction sorbed to airborne particulates (phi): 2.75E-007 (Junge,Mackay)
    Note: the sorbed fraction may be resistant to atmospheric oxidation

 Soil Adsorption Coefficient (PCKOCWIN v1.66):
      Koc    :  1204
      Log Koc:  3.081 

 Aqueous Base/Acid-Catalyzed Hydrolysis (25 deg C) [HYDROWIN v1.67]:
    Rate constants can NOT be estimated for this structure!

 Bioaccumulation Estimates from Log Kow (BCFWIN v2.17):
   Log BCF from regression-based method = 2.719 (BCF = 523.4)
       log Kow used: 4.44 (expkow database)

 Volatilization from Water:
    Henry LC:  0.294 atm-m3/mole  (Henry experimental database)
    Half-Life from Model River:      1.193  hours
    Half-Life from Model Lake :      110.9  hours   (4.62 days)

 Removal In Wastewater Treatment (recommended maximum 95%):
    Total removal:              99.34  percent
    Total biodegradation:        0.11  percent
    Total sludge adsorption:    31.63  percent
    Total to Air:               67.61  percent
      (using 10000 hr Bio P,A,S)

 Level III Fugacity Model:
           Mass Amount    Half-Life    Emissions
            (percent)        (hr)       (kg/hr)
   Air       1.13            1.94         1000       
   Water     38.7            900          1000       
   Soil      34.9            1.8e+003     1000       
   Sediment  25.3            8.1e+003     0          
     Persistence Time: 201 hr


"""

detect_water_sol(text)

[('4.071', '', ''), ('4.071', '', ''), ('', '', '2.49')]
Double value for index 0 detected


[4.071, None, 2.49]